* https://www.kaggle.com/c/fake-news-pair-classification-challenge/data
* https://github.com/fxsjy/jieba

BERT
* https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb#scrollTo=Hcpfl4N2EdOk
* https://github.com/google-research/bert/blob/master/multilingual.md

Gluon-bert
* https://gluon-nlp.mxnet.io/examples/sentence_embedding/bert.html

Embedding options
* https://github.com/Embedding/Chinese-Word-Vectors (Embedding 1)
* https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md (Embedding 2)

Performance
* Using Embedding 1: Performance 0.70640
* Using Embedding 2: Performance 0.68641
* Using BERT: Performance 0.849
* Using BERT_WEIGHTED: Performance 0.85526
* Using BERT_WEIGHTED+SameTID-Agreed: 0.85479
    * Example: df_test[df_test.pred_label != df_test.pred_label2]인 아이들 다 agreed
* Using BERT_WEIGHTED_train100: Performance 0.85634
* Using BERT_ensemble_3models (cn, multi, uncased24): Performance 0.85619
* Using BERT_ensemble_4models_weighted (cn, cnen, multi, uncased24): Performance 0.86283

내일 submission: train / validation을 통해 epoch 최고값 잡고, 여러모델 최종값 merge 

In [1]:
import datetime
import glob
import pandas as pd
import numpy as np
import os
import pickle

In [4]:
## Geomean

method = 'all_average_9_BERT_cn_model'
submission_ensemble_path = '../data/sample_submission_BERT_'+method+'.csv'


smax = {}

# BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12', 'chinese_L-12_H-768_A-12_EN', 'multi_cased_L-12_H-768_A-12', 'uncased_L-24_H-1024_A-16']
BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12']

for BERT_MODEL in BERT_MODEL_LIST: 
    for numrun in range(1,10):
        pkls_path = '../data/sample_submission_sd_BERTopt'+str(numrun)+'_'+BERT_MODEL+'_RESULT_SOFTMAX.p'
        with open(pkls_path, 'rb') as file:
            smax[str(numrun)+'_'+BERT_MODEL] = np.asarray(pickle.load(file))
        
# ens_rslt = np.multiply(smax['chinese_L-12_H-768_A-12', smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16']])   
# ens_rslt = np.average(np.array([smax['chinese_L-12_H-768_A-12'], smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16'], smax['chinese_L-12_H-768_A-12_EN']]), 
#                       axis=0, weights=[0.6,0.1,0.1,0.2])


ens_rslt = np.average(np.array([smax[key] for key in smax.keys()]), axis=0)

test_path = '../data/test.csv'
df_test = pd.read_csv(test_path)
df_test['pred_label'] = np.argmax(ens_rslt, axis=1)
idx_label = {0:'unrelated', 1:'agreed', 2:'disagreed'}
df_test['pred_label'] = df_test['pred_label'].apply(lambda x: idx_label[x])

df_test[['id', 'pred_label']].rename(columns={'id':'Id','pred_label':'Category'}).to_csv(submission_ensemble_path, index=False)